In [ ]:
import os
import psycopg2
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

# Database connection parameters
DB_PARAMS = {
    "dbname": os.getenv("DB_NAME"),
    "user": os.getenv("DB_USER"),
    "password": os.getenv("DB_PASSWORD"),
    "host": os.getenv("DB_HOST"),
    "port": os.getenv("DB_PORT")
}

# Stocks dictionary
#bank names
banks_dict= {'BMO': 'BMO.TO', 
    'CIBC': 'CM.TO', 
    'RBC': 'RY.TO', 
    'Scotiabank': 'BNS.TO', 
    'TD': 'TD.TO'}


try:
    # Connect to the database
    conn = psycopg2.connect(**DB_PARAMS)
    cursor = conn.cursor()

    # Insert query
    insert_query = """
    INSERT INTO stocks (bank_name, ticker) 
    VALUES (%s, %s) 
    ON CONFLICT (ticker) DO NOTHING;
    """

    # Insert each stock into the database
    cursor.executemany(insert_query, banks_dict.items())

    # Commit the transaction
    conn.commit()

    print("Stocks inserted successfully!")

except Exception as e:
    print(f"Error: {e}")

finally:
    cursor.close()
    conn.close()

Stocks inserted successfully!


In [11]:
import os
import pandas as pd
import psycopg2
from dotenv import load_dotenv
import logging

# Load environment variables
load_dotenv()

# Database connection parameters
DB_PARAMS = {
    "dbname": os.getenv("DB_NAME"),
    "user": os.getenv("DB_USER"),
    "password": os.getenv("DB_PASSWORD"),
    "host": os.getenv("DB_HOST"),
    "port": os.getenv("DB_PORT")
}

banks_dict = {
        'BMO': 'BMO.TO', 
        'CIBC': 'CM.TO', 
        'RBC': 'RY.TO', 
        'Scotiabank': 'BNS.TO', 
        'TD': 'TD.TO'
}

# Directory containing the CSV files
csv_directory = "./canadian_banks_stock_data/historical_data"

# Function to insert stock prices into the database
def insert_stock_prices(file_path):
    # Initialize variables before the try block
    conn = None
    cursor = None
    
    try:
        # Extract bank name (ticker) from file name
        file_name = os.path.basename(file_path)
        bank_name = file_name.split('_')[0]  # Get bank name from the file name
        print(bank_name)
        # Look up ticker using bank name
        ticker = banks_dict.get(bank_name, None)
        if ticker is None:
            logging.error(f"Ticker for {bank_name} not found in the dictionary.")
            return

        # Read CSV while skipping the first 3 rows
        df = pd.read_csv(file_path, skiprows=3)

        # Manually set the column headers
        df.columns = ["date", "close_price", "high_price", "low_price", "open_price", "volume"]

        # Prepare data to insert
        stock_prices = df[["date", "close_price", "high_price", "low_price", "open_price", "volume"]].values.tolist()

        # Add ticker to the beginning of each row
        for row in stock_prices:
            row.insert(0, ticker)

        # Connect to the database
        conn = psycopg2.connect(**DB_PARAMS)
        cursor = conn.cursor()

        # Insert stock prices into the database
        insert_query = """
        INSERT INTO stock_prices (stock_ticker, date, close_price, high_price, low_price, open_price, volume)
        VALUES (%s, %s, %s, %s, %s, %s, %s)
        ON CONFLICT (stock_ticker, date) DO NOTHING;
        """

        # Execute insertion for all rows
        cursor.executemany(insert_query, stock_prices)

        # Commit the transaction
        conn.commit()

        print(f"Data from {file_path} inserted successfully!")

    except Exception as e:
        print(f"Error: {e}")
        logging.error(f"Error: {e}")

    finally:
        # Check if cursor and conn were initialized before trying to close them
        if cursor:
            cursor.close()
        if conn:
            conn.close()

# Automate the processing of all CSV files
for file_name in os.listdir(csv_directory):
    if file_name.endswith(".csv"):
        insert_stock_prices(os.path.join(csv_directory, file_name))


BMO
Data from C:/Users/MiF/canadian_banks_stock_data/historical_data\BMO_historical_data.csv inserted successfully!
CIBC
Data from C:/Users/MiF/canadian_banks_stock_data/historical_data\CIBC_historical_data.csv inserted successfully!
